In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from dataImporter import *
from datamodel import TrackSession
import folium
import math
import utils


In [2]:
sourceFile = input("Data file: ")
dataReader = getFileImporter(sourceFile)
session = dataReader.readSessionData()
session.addSessionInfo(sourcefile = sourceFile)
session.trimEnds()

In [ ]:
print("Lap times")
print("---------")
for count,lap in enumerate(session.getLapTimes()):
    print(f"Lap { count }: {math.trunc(lap/60):02}:{lap%60:0>6.3f}")


In [ ]:
print('These datapoints are available:')
for point in session.getDataPointsAvail():
    print (f"- {point}")

In [ ]:
boundingBox = session.getImageBoundaries()
location = session.getMapLocation()
mapPoints = []
m = folium.Map(location=session.getMapLocation(), zoom_start=15, tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr="Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community")
m.fit_bounds(boundingBox)
for lap in session.getLaps():
    mapPoints = []
    for measurement in lap:
        if measurement["segment"] == 1:
            mapPoints.append([measurement["GPSlat"], measurement["GPSlng"]])
    folium.PolyLine(mapPoints).add_to(m)
m


In [5]:
# Generate a map with the fastest segment line in red, others in blue.
m = folium.Map(location=session.getMapLocation(), zoom_start=15, tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr="Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community")
traces = sorted(session.getSegmentsByTime(7), key=lambda x: x['time'])

for trace in traces:
    mapPoints = []
    for pt in trace["path"]:
        mapPoints.append([pt["GPSlat"], pt["GPSlng"]])
    folium.PolyLine(mapPoints).add_to(m)

mapPoints = []
for pt in traces[0]["path"]:
    mapPoints.append([pt["GPSlat"], pt["GPSlng"]])
folium.PolyLine(mapPoints, color="red").add_to(m)


m